In [2]:
import os
import sys
import numpy as np
import pandas as pd

In [58]:
#Load in your data
Customer_Demo = pd.read_excel('customer_demographics.xlsx', keep_default_na=False, na_values=[""])
Customer_Loan = pd.read_excel('customer_loans.xlsx', keep_default_na=False, na_values=[""])


In [59]:
#Check your data
print(Customer_Demo.head())
print("\n---------------------------------------------------------------------------------------------------")
print(Customer_Loan.head())

     custid              Name                                           Job  \
0  92886957  Barbara Thompson  Automotive Service Technicians and Mechanics   
1  16462625        James Wade                               Prosthodontists   
2  89457436      Joel Jackson                                      Surgeons   
3  68462543    Tyler Clements                       Physicians and Surgeons   
4  63315493     Adam Stephens  Railroad Brake, Signal, and Switch Operators   

   Age       Lat       Long  Income  CreditScore  HouseholdSize  ZipCode  \
0   39  31.84568 -102.36764   43415          658              4    79761   
1   93  42.73920  -84.62081  564164          786              4    48917   
2   99  44.27804  -88.27205  466344          676              1    54140   
3  100  33.50921 -111.89903  756171          694              2    85250   
4   97  33.93113 -117.54866   58948          688              2    92860   

      MajorCity State  MedianHomeValue  MedianHouseholdIncome       

In [62]:
#Create your merged df
Merged_Loan_Data = pd.merge(left=Customer_Demo, right=Customer_Loan, left_on='custid', right_on='custid')

In [63]:
print(Merged_Loan_Data.head())

     custid              Name                                           Job  \
0  92886957  Barbara Thompson  Automotive Service Technicians and Mechanics   
1  16462625        James Wade                               Prosthodontists   
2  89457436      Joel Jackson                                      Surgeons   
3  68462543    Tyler Clements                       Physicians and Surgeons   
4  63315493     Adam Stephens  Railroad Brake, Signal, and Switch Operators   

   Age       Lat       Long  Income_x  CreditScore_x  HouseholdSize  ZipCode  \
0   39  31.84568 -102.36764     43415            658              4    79761   
1   93  42.73920  -84.62081    564164            786              4    48917   
2   99  44.27804  -88.27205    466344            676              1    54140   
3  100  33.50921 -111.89903    756171            694              2    85250   
4   97  33.93113 -117.54866     58948            688              2    92860   

   ... CreditScore_y     Debt_y       Date  

In [64]:
#Now to add in localisation
import geopy # Location to ZIp Code
import surgeo #Zip + Name to race

In [65]:
#Lets start assuming Zip Code and lat long are same
fsg = surgeo.BIFSGModel() #Model taking first and last name, zip  

In [66]:
#Prepare First and last Name as well as title and stuff
from nameparser import HumanName

In [67]:
Merged_Loan_Data["title"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).title)
Merged_Loan_Data["first"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).first)
Merged_Loan_Data["middle"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).middle)
Merged_Loan_Data["last"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).last)
Merged_Loan_Data["suffix"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).suffix)
Merged_Loan_Data["nickname"] = Merged_Loan_Data["Name"].apply(lambda x: HumanName(x).nickname)

In [68]:
Merged_Loan_Data.head()



,custid,Name,Job,Age,Lat,Long,Income_x,CreditScore_x,HouseholdSize,ZipCode,...,HomeValue,LoanAmount,ProductType,LoanStatus,title,first,middle,last,suffix,nickname
0,92886957,Barbara Thompson,Automotive Service Technicians and Mechanics,39,31.84568,-102.36764,43415,658,4,79761,...,179930.0,71972.0,Government_insured,Approved,,Barbara,,Thompson,,
1,16462625,James Wade,Prosthodontists,93,42.73920,-84.62081,564164,786,4,48917,...,386400.0,38640.0,Fixed_rate,Approved,,James,,Wade,,
2,89457436,Joel Jackson,Surgeons,99,44.27804,-88.27205,466344,676,1,54140,...,375840.0,75168.0,Government_insured,Approved,,Joel,,Jackson,,
3,68462543,Tyler Clements,Physicians and Surgeons,100,33.50921,-111.89903,756171,694,2,85250,...,675540.0,67554.0,Government_insured,Approved,,Tyler,,Clements,,
4,63315493,Adam Stephens,"Railroad Brake, Signal, and Switch Operators",97,33.93113,-117.54866,58948,688,2,92860,...,731690.0,146338.0,Government_insured,Approved,,Adam,,Stephens,,


In [99]:
fsg_results = fsg.get_probabilities(Merged_Loan_Data["first"], Merged_Loan_Data["last"], Merged_Loan_Data["ZipCode"])

In [100]:
#df[df.columns.difference(['B', 'D'])]
fsg_results['Max'] = fsg_results[fsg_results.columns.difference(['first_name', 'surname', 'zcta5'])].idxmax(axis=1)


In [101]:
print(fsg_results.head())
fsg_results['first_name'] = fsg_results['first_name'].str.capitalize()
fsg_results['surname'] = fsg_results['surname'].str.capitalize()

   zcta5 first_name   surname     white     black       api    native  \
0  79761    BARBARA  THOMPSON  0.780611  0.191603  0.000039  0.004014   
1  48917      JAMES      WADE  0.833131  0.143230  0.000237  0.001976   
2  54140       JOEL   JACKSON  0.969914  0.012615  0.000100  0.004491   
3  85250      TYLER  CLEMENTS  0.987164  0.000207  0.000058  0.002820   
4  92860       ADAM  STEPHENS  0.963161  0.013266  0.000187  0.001638   

   multiple  hispanic    Max  
0  0.006841  0.016891  white  
1  0.020651  0.000774  white  
2  0.006402  0.006478  white  
3  0.009337  0.000414  white  
4  0.013271  0.008477  white  


In [103]:
fsg_results['zcta5']=fsg_results['zcta5'].astype(int)
print(fsg_results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48675 entries, 0 to 48674
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zcta5       48675 non-null  int32  
 1   first_name  48675 non-null  object 
 2   surname     48675 non-null  object 
 3   white       48506 non-null  float64
 4   black       48506 non-null  float64
 5   api         48506 non-null  float64
 6   native      48506 non-null  float64
 7   multiple    48506 non-null  float64
 8   hispanic    48506 non-null  float64
 9   Max         48506 non-null  object 
dtypes: float64(6), int32(1), object(3)
memory usage: 3.5+ MB
None


In [104]:
Merged_Loan_Data_2 = pd.merge(left=Merged_Loan_Data, right=fsg_results, left_on=['first','last','ZipCode'], right_on=['first_name','surname','zcta5'])

In [106]:
Merged_Loan_Data_2.head()

,custid,Name,Job,Age,Lat,Long,Income_x,CreditScore_x,HouseholdSize,ZipCode,...,zcta5,first_name,surname,white,black,api,native,multiple,hispanic,Max
0,92886957,Barbara Thompson,Automotive Service Technicians and Mechanics,39,31.84568,-102.36764,43415,658,4,79761,...,79761,Barbara,Thompson,0.780611,0.191603,0.000039,0.004014,0.006841,0.016891,white
1,16462625,James Wade,Prosthodontists,93,42.73920,-84.62081,564164,786,4,48917,...,48917,James,Wade,0.833131,0.143230,0.000237,0.001976,0.020651,0.000774,white
2,89457436,Joel Jackson,Surgeons,99,44.27804,-88.27205,466344,676,1,54140,...,54140,Joel,Jackson,0.969914,0.012615,0.000100,0.004491,0.006402,0.006478,white
3,68462543,Tyler Clements,Physicians and Surgeons,100,33.50921,-111.89903,756171,694,2,85250,...,85250,Tyler,Clements,0.987164,0.000207,0.000058,0.002820,0.009337,0.000414,white
4,63315493,Adam Stephens,"Railroad Brake, Signal, and Switch Operators",97,33.93113,-117.54866,58948,688,2,92860,...,92860,Adam,Stephens,0.963161,0.013266,0.000187,0.001638,0.013271,0.008477,white
